<a href="https://colab.research.google.com/github/yuga2005/test/blob/master/Sentiment_Analysis_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch
!pip install transformers datasets evaluate accelerate
!pip install vaderSentiment

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.6 MB/s eta 0:00:00


In [1]:
# prompt: write a code for sentiment analysis

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased-finetuned-imdb"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the IMDB dataset
dataset = load_dataset("imdb")

# Preprocess the data
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Evaluate the model
model.eval()

# Vader sentiment analysis
analyzer = SentimentIntensityAnalyzer()

def compute_metrics(eval_pred):
  """Calculates accuracy, F1 score and Vader sentiment analysis."""
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = accuracy_score(labels, predictions)
  f1_score = f1_score(labels, predictions)

  # Vader sentiment analysis
  vader_scores = []
  for text in dataset["test"]["text"]:
    vader_scores.append(analyzer.polarity_scores(text)["compound"])

  return {"accuracy": accuracy, "f1_score": f1_score, "vader_scores": vader_scores}

metrics = model.evaluate(tokenized_dataset["test"], compute_metrics=compute_metrics)

print(f"Accuracy: {metrics['accuracy']}")
print(f"F1 Score: {metrics['f1_score']}")
print(f"Vader Scores: {metrics['vader_scores']}")


ModuleNotFoundError: No module named 'datasets'

In [22]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

# Download NLTK VADER lexicon
nltk.download('vader_lexicon')

# Function to read data from a file
def read_data(file_path):
    # Assuming the file is a CSV with a column 'text' containing the text data
    df = pd.read_csv(file_path, header=None)
    texts = df.values.flatten().tolist()
    print(texts)
    return texts

# Function for NLTK VADER sentiment analysis
def nltk_sentiment_analysis(texts):
    sid = SentimentIntensityAnalyzer()
    results = []
    for text in texts:
        scores = sid.polarity_scores(text)
        results.append(scores)
    return results

# Function for Hugging Face Transformers sentiment analysis
def hf_sentiment_analysis(texts):
    classifier = pipeline('sentiment-analysis')
    results = classifier(texts)
    return results

# Main function
def main(file_path):
    # Read data
    texts = read_data(file_path)

    # Perform NLTK sentiment analysis
    nltk_results = nltk_sentiment_analysis(texts)
    print("NLTK Sentiment Analysis Results:")
    for result in nltk_results:
        print(result)

    # Perform Hugging Face Transformers sentiment analysis
    hf_results = hf_sentiment_analysis(texts)
    print(hf_results)
    print("\nHugging Face Sentiment Analysis Results:")
    for text, result in zip(texts,hf_results):
        print(text,result)

# Example usage
if __name__ == "__main__":
    file_path = 'reviews.txt'  # Replace with your file path
    main(file_path)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


['The new iPhone is incredibly fast and sleek.', 'I am disappointed with the battery life of the iPhone.', 'The camera quality on the iPhone is just amazing!', 'The iPhone is too expensive for the features it offers.', 'I love the new design of the iPhone.', "The iPhone's Face ID works flawlessly.", "I'm not happy with the iPhone's screen durability.", "The iPhone's performance is top-notch.", "I had issues with the iPhone's software updates.", 'The customer service for iPhone repairs is excellent.', "The iPhone's sound quality is impressive.", "I faced problems with the iPhone's connectivity.", 'The iPhone is user-friendly and intuitive.', 'The battery life of the iPhone is outstanding.', "The iPhone's build quality is not up to the mark.", 'I love the variety of apps available for the iPhone.', 'The iPhone overheats quickly when used for gaming.', "The iPhone's display is stunning and vibrant.", 'The iPhone offers great value for money.', "The iPhone's charging speed is very slow.", 

AttributeError: 'list' object has no attribute 'T'

In [11]:
df = pd.read_csv(file_path)
texts = df.values.flatten().tolist()
print(texts)
print(type(texts))

['I am disappointed with the battery life of the iPhone.', 'The camera quality on the iPhone is just amazing!', 'The iPhone is too expensive for the features it offers.', 'I love the new design of the iPhone.', "The iPhone's Face ID works flawlessly.", "I'm not happy with the iPhone's screen durability.", "The iPhone's performance is top-notch.", "I had issues with the iPhone's software updates.", 'The customer service for iPhone repairs is excellent.', "The iPhone's sound quality is impressive.", "I faced problems with the iPhone's connectivity.", 'The iPhone is user-friendly and intuitive.', 'The battery life of the iPhone is outstanding.', "The iPhone's build quality is not up to the mark.", 'I love the variety of apps available for the iPhone.', 'The iPhone overheats quickly when used for gaming.', "The iPhone's display is stunning and vibrant.", 'The iPhone offers great value for money.', "The iPhone's charging speed is very slow.", "I'm not satisfied with the iPhone's camera qual

In [1]:
import pandas as pd
import nltk
import spacy
from transformers import pipeline

# Download required resources
nltk.download('punkt')
nltk.download('stopwords')
spacy.cli.download('en_core_web_sm')

# Load NLP models
nlp = spacy.load('en_core_web_sm')
sentiment_pipeline = pipeline('sentiment-analysis')
summarization_pipeline = pipeline('summarization')

# Example feedback data
feedback_data = [
    "The product is excellent and exceeded my expectations.",
    "I am not satisfied with the service. It was slow and unhelpful.",
    "Great experience! I love how easy it is to use.",
    "The quality could be better. It feels cheap."
]

# Define custom stopwords
custom_stopwords = set(nltk.corpus.stopwords.words('english')).union(set([
    'product', 'service', 'quality', 'experience'
]))

# Preprocessing function with custom stopwords
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha and token.lemma_ not in custom_stopwords]
    return ' '.join(tokens)

# Preprocess feedback data
preprocessed_feedback = [preprocess_text(feedback) for feedback in feedback_data]

# Perform sentiment analysis
sentiment_results = sentiment_pipeline(preprocessed_feedback)

# Print sentiment results
for feedback, sentiment in zip(feedback_data, sentiment_results):
    print(f"Feedback: {feedback}")
    print(f"Sentiment: {sentiment['label']}, Score: {sentiment['score']:.2f}\n")

# Perform summarization using generative AI features
summarized_feedback = summarization_pipeline(' '.join(preprocessed_feedback), max_length=50, min_length=10, do_sample=False)

# Print summarized feedback
print("Summarized Feedback:")
print(summarized_feedback[0]['summary_text'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that aut

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Your max_length is set to 50, but your input_length is only 17. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


Feedback: The product is excellent and exceeded my expectations.
Sentiment: POSITIVE, Score: 1.00

Feedback: I am not satisfied with the service. It was slow and unhelpful.
Sentiment: POSITIVE, Score: 0.94

Feedback: Great experience! I love how easy it is to use.
Sentiment: POSITIVE, Score: 1.00

Feedback: The quality could be better. It feels cheap.
Sentiment: NEGATIVE, Score: 1.00

Summarized Feedback:
 excellent exceed expectation satisfied satisfied satisfied . slow unhelpful slow unhelping great love great love easy use well feel cheap .


In [ ]:
feedbacks = [
    "The new iPhone is incredibly fast and sleek.",
    "I am disappointed with the battery life of the iPhone.",
    "The camera quality on the iPhone is just amazing!",
    "The iPhone is too expensive for the features it offers.",
    "I love the new design of the iPhone.",
    "The iPhone's Face ID works flawlessly.",
    "I'm not happy with the iPhone's screen durability.",
    "The iPhone's performance is top-notch.",
    "I had issues with the iPhone's software updates.",
    "The customer service for iPhone repairs is excellent.",
    "The iPhone's sound quality is impressive.",
    "I faced problems with the iPhone's connectivity.",
    "The iPhone is user-friendly and intuitive.",
    "The battery life of the iPhone is outstanding.",
    "The iPhone's build quality is not up to the mark.",
    "I love the variety of apps available for the iPhone.",
    "The iPhone overheats quickly when used for gaming.",
    "The iPhone's display is stunning and vibrant.",
    "The iPhone offers great value for money.",
    "The iPhone's charging speed is very slow.",
    "I'm not satisfied with the iPhone's camera quality."
]

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
results = classifier(feedbacks)

# Print the results
for feedback, result in zip(feedbacks, results):
    print(f"Feedback: {feedback}")
    print(f"Sentiment Analysis Result: {result}")
    print()

Feedback: The new iPhone is incredibly fast and sleek.
Sentiment Analysis Result: {'label': 'POSITIVE', 'score': 0.999714195728302}

Feedback: I am disappointed with the battery life of the iPhone.
Sentiment Analysis Result: {'label': 'NEGATIVE', 'score': 0.9997648596763611}

Feedback: The camera quality on the iPhone is just amazing!
Sentiment Analysis Result: {'label': 'POSITIVE', 'score': 0.9998635053634644}

Feedback: The iPhone is too expensive for the features it offers.
Sentiment Analysis Result: {'label': 'NEGATIVE', 'score': 0.9995748400688171}

Feedback: I love the new design of the iPhone.
Sentiment Analysis Result: {'label': 'POSITIVE', 'score': 0.999834418296814}

Feedback: The iPhone's Face ID works flawlessly.
Sentiment Analysis Result: {'label': 'POSITIVE', 'score': 0.9998468160629272}

Feedback: I'm not happy with the iPhone's screen durability.
Sentiment Analysis Result: {'label': 'NEGATIVE', 'score': 0.9997497200965881}

Feedback: The iPhone's performance is top-notc

In [ ]:
# prompt: code for sentiment analysis for above feedbacks with gen ai capabilities using transformers

from transformers import pipeline
# !pip install torch
# !pip install transformers datasets evaluate accelerate
feedbacks = [
    "The butter chicken at this restaurant is out of this world!",
    "I was disappointed with the slow service, but the food was decent.",
    "The ambiance is very inviting and the staff is friendly.",
    "The biryani here lacks the authentic flavor I was expecting.",
    "Excellent place for a family dinner, the kids loved the food!",
    "The vegetarian options are plentiful and delicious.",
    "I found a hair in my food and the management was unapologetic.",
    "The samosas are crispy and filled with tasty goodness.",
    "Highly overpriced for the quality of food they serve.",
    "The paneer tikka is a must-try, absolutely delightful!",
    "I had an amazing time, the live music added a great touch.",
    "The cleanliness of the restaurant needs to be improved.",
    "The masala dosa was crispy and full of flavor.",
    "Service was exceptional, and the food came out quickly.",
    "The naan bread was too hard and not fresh.",
    "Great place to host a birthday party, very accommodating staff.",
    "The seafood platter was fresh and well-prepared.",
    "I felt that the portion sizes were too small for the price.",
    "The restaurant has a great selection of Indian sweets.",
    "The air conditioning was not working, making it uncomfortable to dine."
]
classifier = pipeline("sentiment-analysis", model="facebook/bart-large-mnli")
results = classifier(feedbacks)

# Print the results
for feedback, result in zip(feedbacks, results):
    print(f"Feedback: {feedback}")
    print(f"Sentiment Analysis Result: {result}")
    print()


Feedback: The butter chicken at this restaurant is out of this world!
Sentiment Analysis Result: {'label': 'neutral', 'score': 0.9722103476524353}

Feedback: I was disappointed with the slow service, but the food was decent.
Sentiment Analysis Result: {'label': 'neutral', 'score': 0.9962331652641296}

Feedback: The ambiance is very inviting and the staff is friendly.
Sentiment Analysis Result: {'label': 'neutral', 'score': 0.990131139755249}

Feedback: The biryani here lacks the authentic flavor I was expecting.
Sentiment Analysis Result: {'label': 'neutral', 'score': 0.9717035889625549}

Feedback: Excellent place for a family dinner, the kids loved the food!
Sentiment Analysis Result: {'label': 'neutral', 'score': 0.9906031489372253}

Feedback: The vegetarian options are plentiful and delicious.
Sentiment Analysis Result: {'label': 'neutral', 'score': 0.9882915616035461}

Feedback: I found a hair in my food and the management was unapologetic.
Sentiment Analysis Result: {'label': 'neu